#### Prueba de Scraping

In [1]:
pip install selenium beautifulsoup4 pandas webdriver_manager google-cloud-storage 

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import os
import glob
import time
import calendar
import logging
from datetime import datetime, date
from typing import Optional, List, Tuple, Dict, Any

# Importaciones externas
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait, Select
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, WebDriverException

from google.cloud import storage
from google.oauth2 import service_account


In [3]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-
"""
Script para extracción de datos del portal SICOE y carga a Google Cloud Storage.

Este script automatiza la extracción de reportes desde el sistema SICOE y los sube
a un bucket de Google Cloud Storage. Incluye funcionalidades para:
1. Iniciar sesión en el portal SICOE
2. Navegar a la sección de reportes
3. Extraer reportes de ventas normales y cambios
4. Subir los archivos extraídos a Google Cloud Storage
"""


# Configuración del sistema de logs
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
    handlers=[
        logging.FileHandler("sicoe_automation.log"),
        logging.StreamHandler()
    ]
)
logger = logging.getLogger("SICOE_Automation")


class SicoeConfig:
    """Clase para la configuración del script de SICOE."""
    
    def __init__(
        self, 
        login_url: str = "https://sicoe.com.co/sicoe/dist/#/login",
        nit: str = "8301256101",
        username: str = "jefeventas",
        password: str = "Crocs@s2030",
        bucket_name: str = "bucket-quickstart_croc_830",
        destination_prefix: str = "raw/Ventas/sicoe/",
        credentials_path: str = "credentials/croc-454221-e1a3c2e02181.json",
        base_dir: str = "../../",
        file_pattern: str = "*detallado*"
    ):
        """
        Inicializa la configuración para el script de SICOE.
        
        Args:
            login_url: URL de inicio de sesión de SICOE
            nit: Número de identificación tributaria
            username: Nombre de usuario para iniciar sesión
            password: Contraseña para iniciar sesión
            bucket_name: Nombre del bucket de GCS
            destination_prefix: Prefijo/ruta en el bucket donde se subirán los archivos
            credentials_path: Ruta al archivo de credenciales de GCS
            base_dir: Directorio base para buscar los archivos
            file_pattern: Patrón para buscar archivos
        """
        self.login_url = login_url
        self.nit = nit
        self.username = username
        self.password = password
        self.bucket_name = bucket_name
        self.destination_prefix = destination_prefix
        self.credentials_path = credentials_path
        self.base_dir = base_dir
        self.file_pattern = file_pattern
        
    def get_date_range(self) -> Tuple[str, str]:
        """
        Obtiene el rango de fechas del mes actual.
        
        Returns:
            Tupla con el primer y último día del mes en formato 'YYYY-MM-DD'
        """
        today = datetime.now()
        year, month = today.year, today.month
        first_day = date(year, month, 1)
        last_day = date(year, month, calendar.monthrange(year, month)[1])
        
        return (
            first_day.strftime("%Y-%m-%d"), 
            last_day.strftime("%Y-%m-%d")
        )


class WebDriverManager:
    """Clase para gestionar las operaciones del WebDriver."""
    
    def __init__(self, config: SicoeConfig):
        """
        Inicializa el gestor de WebDriver.
        
        Args:
            config: Objeto de configuración con los datos de conexión
        """
        self.config = config
        self.driver = None
        
    def initialize_driver(self) -> webdriver.Edge:
        """
        Inicializa y configura el navegador Edge.
        
        Returns:
            Instancia del WebDriver de Edge configurado
        """
        try:
            options = webdriver.EdgeOptions()
            # Opciones para mejorar el rendimiento y la estabilidad
            options.add_argument("--disable-extensions")
            options.add_argument("--disable-gpu")
            options.add_argument("--no-sandbox")
            options.add_argument("--disable-dev-shm-usage")
            
            self.driver = webdriver.Edge(options=options)
            logger.info("WebDriver de Edge inicializado correctamente")
            return self.driver
        except WebDriverException as e:
            logger.error(f"Error al inicializar el WebDriver: {e}")
            raise
            
    def close_driver(self) -> None:
        """Cierra el navegador y libera recursos."""
        if self.driver:
            try:
                self.driver.quit()
                logger.info("WebDriver cerrado correctamente")
            except Exception as e:
                logger.warning(f"Error al cerrar el WebDriver: {e}")
            finally:
                self.driver = None


class SicoeAutomation:
    """Clase principal para automatizar las operaciones en SICOE."""
    
    def __init__(self, config: SicoeConfig):
        """
        Inicializa la automatización de SICOE.
        
        Args:
            config: Objeto de configuración con los datos de conexión
        """
        self.config = config
        self.driver_manager = WebDriverManager(config)
    
    def login(self, driver: webdriver.Edge) -> bool:
        """
        Realiza el inicio de sesión en el portal SICOE.
        
        Args:
            driver: Instancia del WebDriver
            
        Returns:
            bool: True si el inicio de sesión fue exitoso, False en caso contrario
        """
        try:
            driver.get(self.config.login_url)
            
            # Esperar a que la página se cargue
            WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.ID, "nit"))
            )
            
            # Completar los campos de inicio de sesión
            driver.find_element(By.ID, "nit").send_keys(self.config.nit)
            driver.find_element(By.ID, "login").send_keys(self.config.username)
            driver.find_element(By.ID, "passwd").send_keys(self.config.password)
            
            # Marcar la casilla de verificación y hacer clic en el botón de inicio de sesión
            driver.find_element(By.XPATH, '//*[@id="form"]/div[3]/input').click()
            driver.find_element(By.XPATH, '//*[@id="form"]/div[4]/button').click()
            
            # Esperar a que la página cargue después del inicio de sesión
            WebDriverWait(driver, 20).until(
                EC.presence_of_element_located((By.XPATH, '//*[@id="dock"]/ul/li[5]/a/img'))
            )
            
            logger.info("Inicio de sesión exitoso")
            return True
        except Exception as e:
            logger.error(f"Error durante el inicio de sesión: {e}")
            return False
    
    def navigate_to_report(self, driver: webdriver.Edge) -> bool:
        """
        Navega a la sección de reportes y selecciona el reporte deseado.
        
        Args:
            driver: Instancia del WebDriver
            
        Returns:
            bool: True si la navegación fue exitosa, False en caso contrario
        """
        try:
            # Hacer clic en el botón de informes
            reports_button = driver.find_element(By.XPATH, '//*[@id="dock"]/ul/li[5]/a/img')
            reports_button.click()
            
            # Esperar a que se cargue la sección de informes
            WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.XPATH, '//*[@id="step-1"]/table/tbody/tr/td[2]/a/img'))
            )
            
            # Hacer clic en el informe "Detallado por facturas"
            sales_by_client = driver.find_element(By.XPATH, '//*[@id="step-1"]/table/tbody/tr/td[2]/a/img')
            sales_by_client.click()
            
            logger.info("Navegación a la sección de informes exitosa")
            return True
        except Exception as e:
            logger.error(f"Error al navegar a la sección de informes: {e}")
            return False
    
    def set_date_value(self, driver: webdriver.Edge, field_id: str, date_value: str) -> bool:
        """
        Establece un valor de fecha en un campo de datepicker de solo lectura.
        
        Args:
            driver: Instancia del WebDriver
            field_id: ID del campo de fecha
            date_value: Valor de fecha a establecer (formato: 'YYYY-MM-DD')
            
        Returns:
            bool: True si se estableció el valor correctamente, False en caso contrario
        """
        try:
            date_field = driver.find_element(By.ID, field_id)
            
            # Establecer el valor de fecha utilizando JavaScript
            driver.execute_script("""
                // Establecer el valor directamente
                arguments[0].value = arguments[1];
                
                // Disparar evento de cambio para asegurar que se actualice la validación
                var event = new Event('change', { bubbles: true });
                arguments[0].dispatchEvent(event);
                
                // También disparar el evento de cambio del datepicker si es necesario
                try {
                    if (typeof jQuery !== 'undefined') {
                        jQuery(arguments[0]).datepicker('setDate', arguments[1]);
                    }
                } catch (e) {
                    console.log('Error triggering datepicker:', e);
                }
            """, date_field, date_value)
            
            logger.debug(f"Valor de fecha establecido para {field_id}: {date_value}")
            return True
        except Exception as e:
            logger.error(f"Error al establecer el valor de fecha para {field_id}: {e}")
            return False
    
    def click_excel_button(self, driver: webdriver.Edge, wait_time: int = 3) -> None:
        """
        Hace clic en el botón 'Imprimir Excel' en el formulario modal.
        
        Args:
            driver: Instancia del WebDriver
            wait_time: Tiempo de espera después de hacer clic en el botón (en segundos)
        """
        try:
            excel_button = driver.find_element(By.ID, "excel")
            excel_button.click()
            logger.info("Clic en el botón Excel realizado")
            time.sleep(wait_time)
        except Exception as e:
            logger.error(f"Error al hacer clic en el botón Excel: {e}")
            raise
    
    def handle_report_form(
        self, 
        driver: webdriver.Edge, 
        report_type: str = None,
        wait_time: int = 8
    ) -> bool:
        """
        Maneja el formulario modal de informe y establece los rangos de fechas.
        
        Args:
            driver: Instancia del WebDriver
            report_type: Tipo de informe a generar ('cambio' o None para ventas normales)
            wait_time: Tiempo de espera en segundos después de establecer los valores
            
        Returns:
            bool: True si el manejo del formulario fue exitoso, False en caso contrario
        """
        try:
            # Esperar y cambiar al iframe
            time.sleep(1)
            WebDriverWait(driver, 10).until(
                EC.frame_to_be_available_and_switch_to_it((By.ID, "sb-player"))
            )
            
            # Esperar a que los campos de fecha estén presentes
            WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.ID, "fecha_ini_factura"))
            )
            
            # Obtener rango de fechas del mes actual
            first_day_str, last_day_str = self.config.get_date_range()
            
            # Establecer los valores de fecha
            self.set_date_value(driver, "fecha_ini_factura", first_day_str)
            self.set_date_value(driver, "fecha_fin_factura", last_day_str)
            
            # Si es un informe de tipo "cambio", seleccionar la opción "C"
            if report_type == 'cambio':
                logger.info("Configurando informe de tipo CAMBIO")
                select_element = driver.find_element(By.ID, "id_tipo_producto")
                select = Select(select_element)
                select.select_by_value("C")
                logger.info("Tipo de producto CAMBIO seleccionado correctamente")
            
            # Esperar a que se carguen los datos
            time.sleep(wait_time)
            
            # Hacer clic en el botón Excel
            self.click_excel_button(driver)
            time.sleep(15)
            # Si es un informe de ventas normales, esperar más tiempo para la descarga
            if not report_type:
                time.sleep(25)
            
            # Volver al contenido predeterminado
            driver.switch_to.default_content()
            
            logger.info("Manejo del formulario de informe completado correctamente")
            return True
        except TimeoutException:
            logger.error("Tiempo de espera agotado al esperar el modal o los campos de fecha")
            return False
        except Exception as e:
            logger.error(f"Error al manejar el formulario de informe: {e}")
            return False
    
    def run_report_process(self, report_type: str = None) -> bool:
        """
        Ejecuta el proceso completo para generar un informe.
        
        Args:
            report_type: Tipo de informe a generar ('cambio' o None para ventas normales)
            
        Returns:
            bool: True si el proceso fue exitoso, False en caso contrario
        """
        driver = None
        try:
            # Inicializar el WebDriver
            driver = self.driver_manager.initialize_driver()
            
            # Realizar el inicio de sesión
            if not self.login(driver):
                return False
            
            # Navegar a la sección de informes
            if not self.navigate_to_report(driver):
                return False
            
            # Manejar el formulario de informe
            if not self.handle_report_form(driver, report_type):
                return False
            
            logger.info(f"Proceso de informe {'CAMBIO' if report_type else 'NORMAL'} completado exitosamente")
            return True
        except Exception as e:
            logger.error(f"Error en el proceso de informe: {e}")
            return False
        finally:
            # Cerrar el navegador
            if driver:
                self.driver_manager.close_driver()

    def eliminar_archivos_locales(self) -> int:
        """
        Elimina todos los archivos locales que coinciden con el patrón especificado.
        
        Returns:
            int: Cantidad de archivos eliminados
        """
        try:
            # Buscar archivos que coincidan con el patrón
            patron_path = os.path.join(self.config.base_dir, "**", self.config.file_pattern)
            archivos_encontrados = glob.glob(patron_path, recursive=True)
            
            if not archivos_encontrados:
                logger.warning(f"No se encontraron archivos locales que coincidan con el patrón: {self.config.file_pattern}")
                return 0
            
            # Eliminar archivos encontrados
            count = 0
            for ruta_archivo in archivos_encontrados:
                if os.path.isfile(ruta_archivo):
                    os.remove(ruta_archivo)
                    count += 1
                    logger.info(f"Archivo local eliminado: {ruta_archivo}")
            
            logger.info(f"Total de archivos locales eliminados: {count}")
            return count
        except Exception as e:
            logger.error(f"Error al eliminar archivos locales: {e}")
            raise


class GCSManager:
    """Clase para gestionar las operaciones en Google Cloud Storage."""
    
    def __init__(self, config: SicoeConfig):
        """
        Inicializa el gestor de Google Cloud Storage.
        
        Args:
            config: Objeto de configuración con los datos de conexión a GCS
        """
        self.config = config
        self.credentials = self._get_credentials()
        
    def _get_credentials(self) -> service_account.Credentials:
        """
        Obtiene las credenciales de Google Cloud Storage.
        
        Returns:
            Objeto de credenciales de GCS
        """
        try:
            credentials = service_account.Credentials.from_service_account_file(
                self.config.credentials_path,
                scopes=["https://www.googleapis.com/auth/cloud-platform"],
            )
            logger.debug("Credenciales de GCS obtenidas correctamente")
            return credentials
        except Exception as e:
            logger.error(f"Error al obtener las credenciales de GCS: {e}")
            raise
    
    def eliminar_archivos(self) -> int:
        """
        Elimina todos los archivos en una ruta específica del bucket GCS.
        
        Returns:
            int: Cantidad de archivos eliminados
        """
        try:
            # Crear cliente de almacenamiento
            storage_client = storage.Client(credentials=self.credentials)
            bucket = storage_client.bucket(self.config.bucket_name)
            
            # Listar y eliminar blobs
            blobs = bucket.list_blobs(prefix=self.config.destination_prefix)
            count = 0
            
            for blob in blobs:
                blob.delete()
                count += 1
                logger.info(f"Archivo eliminado: {blob.name}")
            
            logger.info(f"Total de archivos eliminados: {count}")
            return count
        except Exception as e:
            logger.error(f"Error al eliminar archivos de GCS: {e}")
            raise
    
    def subir_archivos(self) -> int:
        """
        Busca archivos que coincidan con un patrón y los sube a GCS.
        
        Returns:
            int: Cantidad de archivos subidos
        """
        try:
            # Crear cliente de almacenamiento
            storage_client = storage.Client(credentials=self.credentials)
            bucket = storage_client.bucket(self.config.bucket_name)
            
            # Buscar archivos que coincidan con el patrón
            patron_path = os.path.join(self.config.base_dir, "**", self.config.file_pattern)
            archivos_encontrados = glob.glob(patron_path, recursive=True)
            
            if not archivos_encontrados:
                logger.warning(f"No se encontraron archivos que coincidan con el patrón: {self.config.file_pattern}")
                return 0
            
            # Subir archivos encontrados
            count = 0
            for ruta_archivo in archivos_encontrados:
                if os.path.isfile(ruta_archivo):
                    # Obtener solo el nombre del archivo sin la ruta completa
                    nombre_archivo = os.path.basename(ruta_archivo)
                    destination_blob_name = f"{self.config.destination_prefix}{nombre_archivo}"
                    
                    # Crear y subir el blob
                    blob = bucket.blob(destination_blob_name)
                    blob.upload_from_filename(ruta_archivo)
                    
                    count += 1
                    logger.info(f"Archivo subido: {ruta_archivo} -> gs://{self.config.bucket_name}/{destination_blob_name}")
            
            logger.info(f"Total de archivos subidos: {count}")
            return count
        except Exception as e:
            logger.error(f"Error al subir archivos a GCS: {e}")
            raise
   

def main():
    """Función principal del script."""
    try:
        
        logger.info("Iniciando proceso de automatización SICOE")
        
        # Crear configuración
        config = SicoeConfig()
        
        # Crear instancias
        sicoe = SicoeAutomation(config)
        gcs_manager = GCSManager(config)
        
        # Ejecutar procesos de informe
        logger.info("Iniciando proceso de informe de CAMBIOS")
        sicoe.run_report_process(report_type='cambio')
        
        logger.info("Iniciando proceso de informe de VENTAS NORMALES")
        sicoe.run_report_process(report_type=None)
        
        # Gestionar archivos en GCS
        logger.info("Iniciando eliminación de archivos en GCS")
        gcs_manager.eliminar_archivos()
        
        logger.info("Iniciando subida de archivos a GCS")
        gcs_manager.subir_archivos()

         # Eliminar archivos locales después de la subida
        logger.info("Iniciando eliminación de archivos locales")
        archivos_eliminados = sicoe.eliminar_archivos_locales()

        logger.info(f"Se eliminaron {archivos_eliminados} archivos locales")
                
        logger.info("Proceso de automatización SICOE completado exitosamente")

    except Exception as e:
        logger.error(f"Error en el proceso principal: {e}", exc_info=True)
    finally:
        logger.info("Finalizando proceso de automatización SICOE")


if __name__ == "__main__":
    main()

2025-08-31 20:26:52,450 - SICOE_Automation - INFO - Iniciando proceso de automatización SICOE
2025-08-31 20:26:52,456 - SICOE_Automation - INFO - Iniciando proceso de informe de CAMBIOS
2025-08-31 20:26:53,089 - SICOE_Automation - ERROR - Error al inicializar el WebDriver: Message: Unable to obtain driver for MicrosoftEdge; For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors/driver_location

2025-08-31 20:26:53,090 - SICOE_Automation - ERROR - Error en el proceso de informe: Message: Unable to obtain driver for MicrosoftEdge; For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors/driver_location

2025-08-31 20:26:53,091 - SICOE_Automation - INFO - Iniciando proceso de informe de VENTAS NORMALES
2025-08-31 20:26:53,216 - SICOE_Automation - ERROR - Error al inicializar el WebDriver: Message: Unable to obtain driver for MicrosoftEdge; For documentation on this